In [69]:
import asyncio
import aiohttp
from bs4 import BeautifulSoup
from pathlib import Path
import requests
from pprint import pprint
from typing import NamedTuple
import pickle
import time
from dotenv import load_dotenv
import os
load_dotenv()

True

In [71]:
class Card(NamedTuple):
    src: str
    title: str

## Getting One Page


In [76]:
URL = os.getenv("POKEMON_PAGE_URL")
page_number = 1
set_res = requests.get(
    URL.format(page_number=page_number))
soup = BeautifulSoup(set_res.text, 'html.parser')
card_imgs = soup.find_all('img', {'class': 'card-image'})

In [77]:
src_sets = [Card(i['srcset'].split(', '), i.parent['title'])
            for i in card_imgs]

card_set = [Card(card.src[1].replace(" 225w", "").strip(), card.title)
            for card in src_sets]
# pprint(card_set[:3])

In [79]:
data_path = (Path.cwd()/'data')
data_path.mkdir(parents=True, exist_ok=True)

try:
    with open(data_path / 'cards.pickle', 'rb') as fp:
        cards = pickle.load(fp)
except EOFError:
    cards = []


with open(data_path / 'saved_index.pickle', 'rb') as fp:
    try:
        START = int(pickle.load(fp)) + 1
    except:
        START = 1

FETCH_NUMBER = 5
for page in range(START, START+FETCH_NUMBER):
    BASE_URL = os.getenv("POKEMON_PAGE_URL")
    try:
        set_res = requests.get(BASE_URL.format(page_number=page))
        soup = BeautifulSoup(set_res.text, 'html.parser')
        card_imgs = soup.find_all('img', {'class': 'card-image'})
        src_sets = [Card(i['srcset'].split(', '), i.parent['title'])
                    for i in card_imgs]
        card_set = [Card(card.src[1].replace(" 225w", "").strip(),
                         card.title) for card in src_sets]

        cards.extend(card_set)

        if (page % 5 == 0):
            with open(data_path / 'cards.pickle', 'wb') as fp:
                pickle.dump(cards, fp)
            with open(data_path / 'saved_index.pickle', 'wb') as fp:
                pickle.dump(page, fp)
        print(page)
    except:
        pass

    time.sleep(0.1)

print(len(cards))
# pprint(cards)

191
192
193
194
195
11640
